In [2]:
import pandas as pd
import requests
from fractions import Fraction
from tqdm import tqdm
from os import listdir
from herbie.archive import Herbie

In [3]:
metar_list = pd.read_csv('Data/BOS.csv')

In [4]:
#this assumes that visibility is always the first value reported in standard miles in the metar
def find_visibility(metar):
    metar_list = metar.split(' ')
    for i,datapoint in enumerate(metar_list):
        if datapoint[-2:] == 'SM':
            if metar_list[i][0] == 'M':
                return float(Fraction(metar_list[i][1:-2]))
            else:
                return float(Fraction(metar_list[i][:-2]))
def find_broken_height(metar):
    metar_list = metar.split(' ')
    for i,datapoint in enumerate(metar_list):
        if datapoint[:3] == 'BKN':
            return 100 * int(metar_list[i][3:6])
    return None

def find_overcast_height(metar):
    metar_list = metar.split(' ')
    for i,datapoint in enumerate(metar_list):
        if datapoint[:3] == 'OVC':
            return 100 * int(metar_list[i][3:6])
    return None

def find_ceiling_height(metar):
    if find_overcast_height(metar) is None and find_broken_height(metar) is None:
        return None
    if find_overcast_height(metar) is None:
        return find_broken_height(metar)
    if find_broken_height(metar) is None:
        return find_overcast_height(metar)
    return min(find_overcast_height(metar), find_broken_height(metar))

def GLAMPstamp(metar):
    metar_list = metar.split(' ')
    datapoint = metar_list[1]
    initalization_time = None
    timestamp = int(datapoint[2:-1])
    if timestamp < 600:
        initalization_time = '00:00'
    elif timestamp < 1200:
        initalization_time = '06:00'
    elif timestamp < 1800:
        initalization_time = '12:00'
    else: 
        initalization_time = '18:00'
    return initalization_time

def format_date(validString):
    dmy = validString.split(' ')[0]
    month, day, year = dmy.split('/')
    if len(day) == 1:
        day = '0' + day
    if len(month) == 1:
        month = '0' + month
    return f'{year}-{month}-{day}'

In [5]:
metar_list['visibility'] = [find_visibility(metar) for metar in metar_list['metar']]
metar_list['ceiling'] = [find_ceiling_height(metar) for metar in metar_list['metar']]
metar_list['ifr_event'] = (metar_list['ceiling'] <= 1000) | (metar_list['visibility'] <= 1)
metar_list['GLAMP timestamp'] = [f'{format_date(metar_list["valid"][i])}T{GLAMPstamp(metar)}Z'
                                 for i, metar in enumerate(metar_list['metar'])]

In [6]:
ifr_list = metar_list[metar_list['ifr_event']]
non_ifr_list = metar_list[~metar_list['ifr_event']]

In [8]:
for time in tqdm(ifr_list['GLAMP timestamp']):   
    if f'{time[:-4]}Z.csv' in listdir('Data/GLAMP IFR data/IFR records'):
        continue
    base_url = 'https://mesonet.agron.iastate.edu/api/1/mos.json'
    params = {'station': 'KBOS',
              'model': 'LAV',
              'runtime': time}
    try:
        response = requests.get(base_url, params=params).json()['data']
        result = pd.DataFrame(response)
        result.to_csv(f'Data/GLAMP IFR data/IFR records/{time[:-4]}Z.csv')
    except KeyError:
        print(time[:-4])

  2%|█▊                                                                               | 66/2942 [00:00<00:08, 323.59it/s]

2020-09-28T06


  4%|███▎                                                                             | 119/2942 [00:01<00:36, 78.13it/s]

2020-09-28T06


 10%|████████▏                                                                       | 301/2942 [00:01<00:10, 262.77it/s]

2020-11-27T00
2020-11-27T00
2020-11-27T00
2020-11-27T00
2020-11-27T00
2020-11-27T00
2020-11-27T00
2020-11-27T00
2020-11-27T00
2020-11-27T00
2020-11-27T00


 13%|██████████▏                                                                      | 370/2942 [00:07<01:28, 29.02it/s]

2020-11-27T00


100%|███████████████████████████████████████████████████████████████████████████████| 2942/2942 [00:14<00:00, 205.84it/s]


In [9]:
for time in tqdm(non_ifr_list['GLAMP timestamp']):   
    if f'{time[:-4]}Z.csv' in listdir('Data/GLAMP IFR data/Non-IFR records'):
        continue
    base_url = 'https://mesonet.agron.iastate.edu/api/1/mos.json'
    params = {'station': 'KBOS',
              'model': 'LAV',
              'runtime': time}
    try:
        response = requests.get(base_url, params=params).json()['data']
        result = pd.DataFrame(response)
        result.to_csv(f'Data/GLAMP IFR data/Non-IFR records/{time[:-4]}Z.csv')
    except KeyError:
        print(time[:-4])

  5%|████                                                                            | 887/17675 [01:38<31:53,  8.77it/s]

2020-09-28T06


  5%|████                                                                            | 888/17675 [01:38<37:00,  7.56it/s]

2020-09-28T06


  5%|████                                                                            | 889/17675 [01:38<44:21,  6.31it/s]

...

2020-09-28T06


  5%|███▉                                                                          | 894/17675 [01:41<1:33:28,  2.99it/s]

2020-09-28T06


 15%|███████████▍                                                                   | 2571/17675 [04:57<29:06,  8.65it/s]


OSError: Could not find a suitable TLS CA certificate bundle, invalid path: /home/bparazin/anaconda3/envs/nws/lib/python3.10/site-packages/certifi/cacert.pem

2020-09-28T06:00Z and 2020-11-27T00:00Z seem to be lost

In [8]:
base_url = 'https://mesonet.agron.iastate.edu/api/1/mos.json'
params = {'station': 'KBOS',
          'model': 'LAV',
          'runtime': '2020-11-27T00:00Z'}
response = requests.get(base_url, params=params)
print(response)
result = pd.DataFrame(response)
#result.to_csv(f'Data/GLAMP IFR data/{time[:-4]}Z.csv')

<Response [404]>


In [5]:
H2 = Herbie("2021-07-21", model="hrrr", product="prs", save_dir='Data')

✅ Found ┊ model=hrrr ┊ product=prs ┊ 2021-Jul-21 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


In [ ]:
H2.download()

👨🏻‍🏭 Created directory: [C:\Users\bpara\Documents\Git\nwsFlightCategories\Data\hrrr\20210721]


In [ ]:
H2_path = 'Data/hrrr/20210721/hrrr.t00z.wrfprsf00.grib2'